# CASO JÓVENES

Queremos resolver dos preguntas relacionadas con los jóvenes (clientes entre 18 y 29 años):
1ª Pregunta: ¿cómo son?
2ª Pregunta. ¿cómo han evolucionado respecto a Noviembre 2020?

### Abrir el fichero de datos

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('C:/Users/nicol/OneDrive/Documentos/VSCLocal/Data/datos/006/3. CASO JÓVENES.xlsx - BASE CLIENTES NOV21.csv')
df.dtypes

CCLIEN                int64
COD EDAD              int64
COD_TERRIT            int64
SALDO CUENTA          int64
IMPORTE CONSUMO       int64
SEGURO COCHE          int64
IMPORTE HIPOTECA    float64
TENENCIA NOMINA       int64
XTI_IDEFISCO          int64
dtype: object

#### Filtros
Es un problema de clientes particulares, así tendremos que filtrar a las personas jurídicas, que si comprobamos en el catálogo, se seleccionan por XTI_IDEFISCO = 2 Ó 3

In [3]:
print('Número de registros: ', df.shape[0])
df_filtro_partic = df[(df['XTI_IDEFISCO'] != 2) & (df['XTI_IDEFISCO'] != 3)]
print('Número de registros filtrando por particulares: ', df_filtro_partic.shape[0])

Número de registros:  1000
Número de registros filtrando por particulares:  915


#### Limpieza de datos
Comprobamos si hay missing values, sustituyéndolos por 0

In [4]:
df_filtro_partic.isna().sum()

CCLIEN              0
COD EDAD            0
COD_TERRIT          0
SALDO CUENTA        0
IMPORTE CONSUMO     0
SEGURO COCHE        0
IMPORTE HIPOTECA    4
TENENCIA NOMINA     0
XTI_IDEFISCO        0
dtype: int64

In [5]:
df_limpio = df_filtro_partic.fillna(0)

In [6]:
df_limpio.isna().sum()

CCLIEN              0
COD EDAD            0
COD_TERRIT          0
SALDO CUENTA        0
IMPORTE CONSUMO     0
SEGURO COCHE        0
IMPORTE HIPOTECA    0
TENENCIA NOMINA     0
XTI_IDEFISCO        0
dtype: int64

#### Variables contador
* Generamos variables contador de tenencia de productos y de hipoteca
* Generamos variable de tramo de edad: menores, jóvenes, entre 30 y 59 y mayores de 59 años

In [7]:
import numpy as np

df_limpio['HIPOTECA'] = np.where(df_limpio['IMPORTE HIPOTECA'] > 0, 1, 0)
df_limpio['CONSUMO'] = np.where(df_limpio['IMPORTE CONSUMO'] > 0, 1, 0)

In [9]:
conditions = [
    (df_limpio['COD EDAD'] < 18),
    (df_limpio['COD EDAD'] >= 18) & (df_limpio['COD EDAD'] < 30),
    (df_limpio['COD EDAD'] >= 30) & (df_limpio['COD EDAD'] <= 59),
    (df_limpio['COD EDAD'] > 59),
]

choices = ['A', 'B', 'D', 'C']

df_limpio['TRAMO_EDAD'] = np.select(conditions, choices, default=0)
df_limpio['TRAMO_EDAD']

0      B
1      C
2      B
3      B
4      B
      ..
995    B
996    D
997    B
998    D
999    B
Name: TRAMO_EDAD, Length: 915, dtype: object

In [10]:
df_limpio.groupby('TRAMO_EDAD').agg({'TRAMO_EDAD': 'count',
                                     'TENENCIA NOMINA': 'sum',
                                     'IMPORTE CONSUMO':'sum',
                                     'CONSUMO': 'sum',
                                     'IMPORTE HIPOTECA':'sum',
                                     'HIPOTECA': 'sum',
                                     'SEGURO COCHE': 'sum',
                                     'SALDO CUENTA': 'sum'})

,TRAMO_EDAD,TENENCIA NOMINA,IMPORTE CONSUMO,CONSUMO,IMPORTE HIPOTECA,HIPOTECA,SEGURO COCHE,SALDO CUENTA
TRAMO_EDAD,,,,,,,,
A,57,0,0,0,0.0,0,0,96100
B,426,136,1283594,237,9299289.0,55,118,2014668
C,157,21,252730,50,3739769.0,19,36,8259325
D,275,142,847728,165,24092870.0,140,96,13186452


In [11]:
df_limpio.groupby('TRAMO_EDAD').agg({'TRAMO_EDAD': lambda x: x.count() / df_limpio.shape[0] * 100,
                                     'TENENCIA NOMINA': lambda x: x.sum() / float(x.count()) * 100,
                                     'CONSUMO': lambda x: x.sum() / float(x.count()) * 100,
                                     'HIPOTECA': lambda x: x.sum() / float(x.count()) * 100,
                                     'SEGURO COCHE': lambda x: x.sum() / float(x.count()) * 100,
                                     'SALDO CUENTA': lambda x: x.sum() / float(x.count())})

,TRAMO_EDAD,TENENCIA NOMINA,CONSUMO,HIPOTECA,SEGURO COCHE,SALDO CUENTA
TRAMO_EDAD,,,,,,
A,6.229508,0.000000,0.000000,0.000000,0.000000,1685.964912
B,46.557377,31.924883,55.633803,12.910798,27.699531,4729.267606
C,17.158470,13.375796,31.847134,12.101911,22.929936,52607.165605
D,30.054645,51.636364,60.000000,50.909091,34.909091,47950.734545


* 1ª Conclusión: los jóvenes suponen un 46,56% de mi base de clientes
* 2ª Conclusión: destaca su tenencia de consumo, así como su tenencia de seguro de coche

#### Cruce de las bases de datos

Vamos a calcular cómo ha evolucionado mi base de jóvenes respecto a Noviembre 2020 (sin filtros)

In [12]:
df_2021 = pd.read_csv('C:/Users/nicol/OneDrive/Documentos/VSCLocal/Data/datos/006/3. CASO JÓVENES.xlsx - BASE CLIENTES NOV21.csv')
print(df_2021.shape)

df_2020 = pd.read_csv('C:/Users/nicol/OneDrive/Documentos/VSCLocal/Data/datos/006/3. CASO JÓVENES.xlsx - BASE CLIENTES NOV20.csv')
print(df_2020.shape)

(1000, 9)
(949, 9)


In [13]:
result = pd.merge(
    df_2021,
    df_2020,
    how="inner",
    on="CCLIEN")
print("Numero de clientes que cruzan: ", result.shape[0])

Numero de clientes que cruzan:  889


1) En primer lugar vamos a ver cómo ha evolucionado la base completa de clientes y posteriormente veremos cómo ha evolucionado la base de jóvenes

In [37]:
print(f"ALTAS: {df_2021.shape[0]-result.shape[0]}")
print(f"BAJAS: {df_2020.shape[0]-result.shape[0]}")
print(f"PERMANECEN: {result.shape[0]}")

ALTAS: 111
BAJAS: 60
PERMANECEN: 889


In [38]:
result

,CCLIEN,COD EDAD_x,COD_TERRIT_x,SALDO CUENTA_x,IMPORTE CONSUMO_x,SEGURO COCHE_x,IMPORTE HIPOTECA_x,TENENCIA NOMINA_x,XTI_IDEFISCO_x,COD EDAD_y,COD_TERRIT_y,SALDO CUENTA_y,IMPORTE CONSUMO_y,SEGURO COCHE_y,IMPORTE HIPOTECA_y,TENENCIA NOMINA_y,XTI_IDEFISCO_y,JOVEN
0,75328,22,72,9396,3006,0,0.0,0,1,21,72,10801,2442,0,150000,0,1,1
1,66872,34,63,24124,2644,1,309954.0,1,1,33,63,27731,2148,1,272524,1,1,0
2,17769,25,63,4097,0,0,239255.0,0,1,24,63,4709,5000,0,210362,0,1,1
3,68575,28,51,7773,2082,0,0.0,0,1,27,51,8935,1691,0,150000,0,1,1
4,50321,32,1525,56809,7737,0,0.0,0,1,31,1525,65304,6287,0,150000,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,71468,19,51,5363,2099,0,0.0,0,1,18,51,6165,1705,0,0,0,1,1
885,17594,46,8302,19344,0,1,0.0,0,1,45,8302,22236,0,1,0,0,1,0
886,70253,22,72,7697,0,1,0.0,0,1,21,72,8848,0,1,0,0,1,1
887,77577,44,1523,91195,4195,1,280510.0,1,1,43,1523,104832,3408,1,246635,1,1,0


Se cumple el cálculo: Base Nov20 - Bajas = Clientes que permanecen o Base Nov21 - Altas = Clientes que permanecen

2) Vamos a hacer el mismo cálculo con los Jóvenes

In [36]:
import numpy as np
df_2021['JOVEN']=np.where((df_2021['COD EDAD'] >= 18) & (df_2021['COD EDAD'] < 30) & (df_2021['XTI_IDEFISCO'] !=2) & (df_2021['XTI_IDEFISCO'] !=3), 1, 0)
df_2020['JOVEN']=np.where((df_2020['COD EDAD'] >= 18) & (df_2020['COD EDAD'] < 30) & (df_2020['XTI_IDEFISCO'] !=2) & (df_2020['XTI_IDEFISCO'] !=3), 1, 0)
result['JOVEN']=np.where((result['COD EDAD_x'] >= 18) & (result['COD EDAD_x'] < 30) & (result['XTI_IDEFISCO_x'] !=2) & (result['XTI_IDEFISCO_x'] !=3), 1, 0)

df_2021_jovenes=df_2021[df_2021['JOVEN']==1]
df_2020_jovenes=df_2020[df_2020['JOVEN']==1]
result_jovenes=result[result['JOVEN']==1]

print(f"JOVENES - 2021: {df_2021_jovenes.shape[0]}")
print(f"JOVENES - 2020: {df_2020_jovenes.shape[0]}")
print(f"JOVENES - PERMANECEN: {result_jovenes.shape[0]}")
print(f"ALTAS: {df_2021_jovenes.shape[0]-result_jovenes.shape[0]}")
print(f"BAJAS: {df_2020_jovenes.shape[0]-result_jovenes.shape[0]}")

JOVENES - 2021: 426
JOVENES - 2020: 396
JOVENES - PERMANECEN: 389
ALTAS: 37
BAJAS: 7


In [45]:
df_all=df_2020.merge(df_2021,on=['CCLIEN'],
                     how='outer',indicator=True)
print(f"Número de clientes que cruzan: {df_all.shape[0]}")
df_all['JOVEN_x']=np.where((df_all['COD EDAD_x'] >= 18) & (df_all['COD EDAD_x'] < 30) & (df_all['XTI_IDEFISCO_x'] !=2) & (df_all['XTI_IDEFISCO_x'] !=3), 1, 0)
df_x=df_all[df_all['JOVEN_x']==1]
df_x.groupby('_merge').agg({'_merge':'count'})


Número de clientes que cruzan: 1060


C:\Users\nicol\AppData\Local\Temp\ipykernel_17892\4036926657.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_x.groupby('_merge').agg({'_merge':'count'})


,_merge
_merge,
left_only,30
right_only,0
both,366


In [46]:
df_all=df_2020.merge(df_2021,on=['CCLIEN'],
                     how='outer',indicator=True)
print(f"Número de clientes que cruzan: {df_all.shape[0]}")
df_all['JOVEN_y']=np.where((df_all['COD EDAD_y'] >= 18) & (df_all['COD EDAD_y'] < 30) & (df_all['XTI_IDEFISCO_y'] !=2) & (df_all['XTI_IDEFISCO_y'] !=3), 1, 0)
df_x=df_all[df_all['JOVEN_y']==1]
df_x.groupby('_merge').agg({'_merge':'count'})

Número de clientes que cruzan: 1060


C:\Users\nicol\AppData\Local\Temp\ipykernel_17892\3883352370.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_x.groupby('_merge').agg({'_merge':'count'})


,_merge
_merge,
left_only,0
right_only,37
both,389


¿Por qué hay dos valores posibles para clientes que permanecen?¿por qué no se cumplen los cálculos anteriores?
Porque nos falta considerar las altas vegetativas / bajas vegetativas: los que cumplen 18 o cumplen 30

In [48]:
df_bajas_vegetativas=df_all[(df_all['COD EDAD_x'] < 30) & (df_all['COD EDAD_y'] >= 30) & (df_all['XTI_IDEFISCO_y'] !=2) & (df_all['XTI_IDEFISCO_y'] !=3)]
print(f"BAJAS VEGETATIVAS: {df_bajas_vegetativas.shape[0]}")

BAJAS VEGETATIVAS: 8


In [49]:
df_altas_vegetativas=df_all[(df_all['COD EDAD_x'] < 18) & (df_all['COD EDAD_y'] >= 18) & (df_all['XTI_IDEFISCO_y'] !=2) & (df_all['XTI_IDEFISCO_y'] !=3)]
print(f"ALTAS VEGETATIVAS: {df_altas_vegetativas.shape[0]}")

ALTAS VEGETATIVAS: 31


Resolución: el buen crecimiento de la base de jóvenes se debe fundamentalmente a las altas vegetativas ya que hay un volumen de altas similar al de bajas, pero más altas vegetativas que bajas vegetativas. Esto tiene un riesgo: si no se cuida la base de menores, este crecimiento no se seguirá produciendo